<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 11:06:26
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: -35.26 K (-0.26%)
Current PnL: -13.48 L (-9.4%)
CY Booked + Current PnL: -6.52 L (-4.55%)
-------------------
Total profit:  4.66 L
Total loss:  -18.15 L
-------------------
Total Booked + Current PnL: 20.46 L (17.55%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.95 L (52.66%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.19,H-LC,91.67,114240.0,10674.0,10030.0,0.00,10.31,8.78,20.00,32.0,1.06,0.84,25.83,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.90,H-LC,83.33,110736.0,6424.0,11107.0,-1.05,6.16,10.03,16.81,27.0,0.58,0.81,23.19,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,171823.0,30983.0,13505.0,-0.69,22.00,7.86,31.59,88.0,2.29,1.26,49.25,XY24,NTT,BANKS
92,VIPIND,488.80,489.00,-211.87,H-SC,97.92,80730.0,-14586.0,14628.0,-0.19,-15.30,18.12,0.04,157.0,-1.00,0.59,59.23,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,33.33,213988.0,12508.0,14701.0,0.08,6.21,6.87,13.51,77.0,0.85,1.57,19.59,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,8.33,179469.0,-339.0,112742.0,-0.88,-0.19,62.82,62.52,173.0,-0.00,1.31,33.83,XR,NTT,BANKS
60,NATIONALUM,189.63,247.44,-50.15,H-MC,54.17,99831.0,-673.0,31317.0,-0.11,-0.67,31.37,30.49,97.0,-0.02,0.73,31.52,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,TRIDENT,37.35,48.00,27.62,M-SC,82.29,80275.0,-11980.0,38283.0,4.43,-12.99,47.69,28.51,233.0,-0.31,0.59,37.36,XR,NTT,TEXTILES
16,BLUSPRING,226.45,85.12,NaN,NaN,53.12,18641.0,-30952.0,0.0,4.22,-62.41,0.00,-62.41,270.0,-inf,0.14,25.01,XY24,ATH,MISC
32,ENRIN,1377.95,3086.10,NaN,NaN,95.83,77152.0,42703.0,0.0,2.87,123.96,0.00,123.96,274.0,inf,0.57,17.98,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.20,-12.15,L-SC,100.00,176299.0,34457.0,22108.0,2.78,24.29,12.54,39.88,262.0,1.56,1.29,78.66,XY24,NTT,POWER
13,BATAINDIA,1550.24,2096.00,-30.58,M-SC,67.71,103874.0,-24796.0,70094.0,1.89,-19.27,67.48,35.20,205.0,-0.35,0.76,5.98,X40,NTT,FOOTWEAR


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,RELAXO,902.64,1176.0,-30.67,H-SC,92.71,62726.0,-52812.0,87804.0,-3.22,-45.71,139.98,30.28,131.0,-0.60,0.46,22.54,X40N,NTT,FOOTWEAR
24,COFFEEDAY,59.14,80.0,-57.75,L-SC,43.75,63360.0,-50189.0,90237.0,-3.17,-44.20,142.42,35.27,265.0,-0.56,0.46,55.08,XR,NTT,HOTELS
22,CIPLA,1495.00,1795.0,-22.84,H-LC,31.25,99294.0,-871.0,20971.0,-1.89,-0.87,21.12,20.07,30.0,-0.04,0.73,7.03,X40N,BTT,PHARMA
11,BANDHANBNK,214.25,400.0,-32.36,H-SC,7.29,201991.0,-46539.0,262003.0,-1.89,-18.73,129.71,86.70,134.0,-0.18,1.48,33.63,XY24,NTT,BANKS
27,DEN,52.79,75.0,120.21,L-SC,80.21,61760.0,-22704.0,58240.0,-1.88,-26.88,94.30,42.07,261.0,-0.39,0.45,31.16,AR,NTT,ENTERTAINMENT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,23.96,127116.0,-15363.0,15978.0,0.17,-10.78,12.57,0.44,247.0,-0.96,0.93,50.03,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.37,M-SC,26.04,150663.0,-20478.0,20430.0,-0.12,-11.97,13.56,-0.03,191.0,-1.00,1.10,8.29,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,94.79,148961.0,-25598.0,25636.0,0.04,-14.66,17.21,0.02,133.0,-1.00,1.09,29.17,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-211.87,H-SC,97.92,80730.0,-14586.0,14628.0,-0.19,-15.30,18.12,0.04,157.0,-1.00,0.59,59.23,OX40N,NTT,MISC
53,KANSAINER,299.63,340.0,-67.65,H-SC,46.88,225315.0,-44352.0,80685.0,0.24,-16.45,35.81,13.47,137.0,-0.55,1.65,12.78,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.70,X-LC,38.54,264865.0,-31069.0,124990.0,-0.47,-10.50,47.19,31.73,1.0,-0.25,1.94,6.02,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,73.96,290893.0,29292.0,116328.0,-0.12,11.20,39.99,55.67,2.0,0.25,2.13,17.99,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-14.11,X-LC,61.46,269203.0,-1144.0,35777.0,-0.28,-0.42,13.29,12.81,5.0,-0.03,1.97,12.59,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.58,X-LC,71.88,269514.0,40281.0,27005.0,-0.42,17.57,10.02,29.35,10.0,1.49,1.97,28.06,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.21,X-LC,28.12,82658.0,-27514.0,116589.0,0.86,-24.97,141.05,80.85,11.0,-0.24,0.61,21.74,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3086.10,NaN,NaN,95.83,77152.0,42703.0,0.0,2.87,123.96,0.00,123.96,274.0,inf,0.57,17.98,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,36.46,50589.0,9334.0,1472.0,1.09,22.62,2.91,26.19,272.0,6.34,0.37,19.10,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-12.15,L-SC,100.00,176299.0,34457.0,22108.0,2.78,24.29,12.54,39.88,262.0,1.56,1.29,78.66,XY24,NTT,POWER
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,171823.0,30983.0,13505.0,-0.69,22.00,7.86,31.59,88.0,2.29,1.26,49.25,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,48.96,230903.0,5941.0,161724.0,0.46,2.64,70.04,74.53,189.0,0.04,1.69,34.06,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.97,24.70,H-SC,32.29,127588.0,2291.0,56866.0,-1.30,1.83,44.57,47.22,143.0,0.04,0.93,59.62,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.09,H-SC,19.79,114694.0,12690.0,32332.0,-1.11,12.44,28.19,44.14,126.0,0.39,0.84,30.70,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,171823.0,30983.0,13505.0,-0.69,22.00,7.86,31.59,88.0,2.29,1.26,49.25,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-2.86,M-LC,88.54,166005.0,15060.0,94390.0,0.00,9.98,56.86,72.51,71.0,0.16,1.22,14.35,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3086.10,NaN,NaN,95.83,77152.0,42703.0,0.0,2.87,123.96,0.00,123.96,274.0,inf,0.57,17.98,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-39.19,M-SC,56.25,200611.0,28990.0,133085.0,-0.59,16.89,66.34,94.43,201.0,0.22,1.47,48.06,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,88.54,166005.0,15060.0,94390.0,0.00,9.98,56.86,72.51,71.0,0.16,1.22,14.35,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,66.67,134530.0,11194.0,122570.0,-0.00,9.08,91.11,108.45,123.0,0.09,0.99,33.46,AR,ATH,MISC
54,KPIGREEN,497.21,731.97,24.70,H-SC,32.29,127588.0,2291.0,56866.0,-1.30,1.83,44.57,47.22,143.0,0.04,0.93,59.62,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,85.12,NaN,NaN,53.12,18641.0,-30952.0,0.0,4.22,-62.41,0.00,-62.41,270.0,-inf,0.14,25.01,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,43.75,63360.0,-50189.0,90237.0,-3.17,-44.20,142.42,35.27,265.0,-0.56,0.46,55.08,XR,NTT,HOTELS
32,ENRIN,1377.95,3086.10,NaN,NaN,95.83,77152.0,42703.0,0.0,2.87,123.96,0.00,123.96,274.0,inf,0.57,17.98,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-17.34,H-SC,77.08,93855.0,-4125.0,25998.0,1.20,-4.21,27.70,22.32,145.0,-0.16,0.69,36.12,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.80,X-LC,41.67,98108.0,363.0,37036.0,-0.21,0.37,37.75,38.26,18.0,0.01,0.72,31.00,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-12.15,L-SC,100.00,176299.0,34457.0,22108.0,2.78,24.29,12.54,39.88,262.0,1.56,1.29,78.66,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,9.11,X-SC,98.96,101523.0,560.0,93970.0,-1.54,0.55,92.56,93.63,67.0,0.01,0.74,51.22,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-211.87,H-SC,97.92,80730.0,-14586.0,14628.0,-0.19,-15.30,18.12,0.04,157.0,-1.00,0.59,59.23,OX40N,NTT,MISC
15,BLUESTARCO,1646.70,2337.55,-3.74,H-SC,96.88,180510.0,15840.0,53250.0,-1.18,9.62,29.50,41.95,124.0,0.30,1.32,17.80,X40N,ATH,AC
32,ENRIN,1377.95,3086.10,NaN,NaN,95.83,77152.0,42703.0,0.0,2.87,123.96,0.00,123.96,274.0,inf,0.57,17.98,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.90
1,25,41.85
2,50,70.55


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.47
LC    33.96
MC    21.49
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      12.93
XY25     12.03
X40N     11.91
XR       10.05
OX40N     8.19
AR        8.19
MH        1.66
X5K       1.45
X200      1.41
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.45
M-SC    15.92
H-LC    14.11
X-LC    12.93
H-MC     9.50
X-MC     7.59
M-LC     5.88
X-SC     4.43
M-MC     4.04
L-SC     2.67
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.23,-0.65,34.30
IT,12.09,-10.08,64.14
FINANCE,8.50,-12.39,53.18
BANKS,7.70,-8.44,52.68
MISC,5.85,-20.52,76.61
PAINTS,5.70,-12.00,34.08
ELECTRICAL,5.48,1.41,45.00
HEALTHCARE,4.63,-1.86,31.61
AUTO,4.30,-15.16,65.80


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2662873.0,26
XR,1086038.0,13
AR,1029544.0,10
X40,689482.0,10
X40N,539440.0,11
XY25,474541.0,8
OX40N,311080.0,11
SR,176565.0,2
MH,88183.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1991201.0,20
M-SC,1767920.0,21
H-LC,615380.0,12
X-LC,544602.0,8
H-MC,536626.0,8
M-MC,417035.0,3
X-MC,416731.0,6
X-SC,311283.0,5
M-LC,259256.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      891946.0      6
M-SC       XY24      810079.0      7
H-SC       AR        526649.0      3
M-SC       XR        334670.0      4
M-MC       XY24      304293.0      2
H-MC       XY24      300399.0      3
X-LC       X40       278354.0      3
H-SC       XR        215068.0      3
           X40N      212418.0      4
H-LC       X40N      197888.0      4
M-SC       AR        184966.0      2
           OX40N     164227.0      5
X-MC       X40       156582.0      2
H-LC       X40       155794.0      3
L-SC       XR        152234.0      2
M-LC       XY24      150165.0      2
X-SC       XR        123041.0      1
M-SC       XY25      121289.0      1
X-LC       AR        116589.0      1
M-MC       XR        112742.0      1
X-MC       XY25      104047.0      1
X-LC       XY25      101969.0      3
M-LC       XR         94390.0      1
X-SC       SR         93970.0      1
H-SC       OX40N      88254.0      3
X-MC       XY24       86461.0      1
H-MC       XY25       85071.0      1
M-SC       SR         82595.0      1
H-LC       AR         80880.0      1
M-SC       X40        70094.0      1
H-LC       X5K        70090.0      2
           X200       67609.0      1
H-MC       AR         62220.0      1
L-SC       AR         58240.0      1
H-SC       MH         56866.0      1
L-MC       XR         53893.0      1
X-SC       XY24       52831.0      1
X-MC       X40N       52483.0      1
X-LC       X40N       47690.0      1
L-LC       XY25       47464.0      1
H-LC       XY24       43119.0      1
X-SC       OX40N      41441.0      2
H-MC       MH         31317.0      1
           X40N       28961.0      1
           X40        28658.0      1
L-SC       XY24       22108.0      1
X-MC       OX40N      17158.0      1
M-LC       XY25       14701.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
